# MLE and MAP Estimation

In this short tutorial we review how to do Maximum Likelihood (MLE) and Maximum a Posteriori (MAP) estimation in Pyro.

In [1]:
import torch
from torch.distributions import constraints
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO

We consider the simple "fair coin" example covered in a [previous tutorial](http://pyro.ai/examples/svi_part_i.html#A-simple-example).

In [2]:
data = torch.zeros(10)
data[0:6] = 1.0

def original_model(data):
    f = pyro.sample("latent_fairness", dist.Beta(10.0, 10.0))
    with pyro.plate("data", data.size(0)):
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

To facilitate comparison between different inference techniques, we construct a training helper:

In [3]:
def train(model, guide, lr=0.005, n_steps=201):
    pyro.clear_param_store()
    adam = pyro.optim.Adam({"lr": lr})
    svi = SVI(model, guide, adam, loss=Trace_ELBO())

    for step in range(n_steps):
        loss = svi.step(data)
        if step % 50 == 0:
            print('[iter {}]  loss: {:.4f}'.format(step, loss))

## MLE

Our model has a single latent variable `latent_fairness`. To do Maximum Likelihood Estimation we simply "demote" our latent variable `latent_fairness` to a Pyro parameter.

In [4]:
def model_mle(data):
    # note that we need to include the interval constraint; 
    # in original_model() this constraint appears implicitly in 
    # the support of the Beta distribution.
    f = pyro.param("latent_fairness", torch.tensor(0.5), 
                   constraint=constraints.unit_interval)
    with pyro.plate("data", data.size(0)):
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

Since we no longer have any latent variables, our guide can be empty:

In [5]:
def guide_mle(data):
    pass

Let's see what result we get.

In [6]:
train(model_mle, guide_mle)

[iter 0]  loss: 6.9315
[iter 50]  loss: 6.7693
[iter 100]  loss: 6.7333
[iter 150]  loss: 6.7302
[iter 200]  loss: 6.7301


In [7]:
print("Our MLE estimate of the latent fairness is {:.3f}".format(
      pyro.param("latent_fairness").item()))

Our MLE estimate of the latent fairness is 0.600


Thus with MLE we get a point estimate of `latent_fairness`.

## MAP

With Maximum a Posteriori estimation, we also get a point estimate of our latent variables. The difference to MLE is that these estimates will be regularized by the prior.

To do MAP in Pyro we use a [Delta distribution](http://docs.pyro.ai/en/stable/distributions.html#pyro.distributions.Delta) for the guide. Recall that the `Delta` distribution puts all its probability mass at a single value. The `Delta` distribution will be parameterized by a learnable parameter. 

In [8]:
def guide_map(data):
    f_map = pyro.param("f_map", torch.tensor(0.5),
                       constraint=constraints.unit_interval)
    pyro.sample("latent_fairness", dist.Delta(f_map))

Let's see how this result differs from MLE.

In [9]:
train(original_model, guide_map)

[iter 0]  loss: 5.6719
[iter 50]  loss: 5.6007
[iter 100]  loss: 5.6004
[iter 150]  loss: 5.6004
[iter 200]  loss: 5.6004


In [10]:
print("Our MAP estimate of the latent fairness is {:.3f}".format(
      pyro.param("f_map").item()))

Our MAP estimate of the latent fairness is 0.536


To understand what's going on note that the prior mean of the `latent_fairness` in our model is 0.5, since that is the mean of `Beta(10.0, 10.0)`. The MLE estimate (which ignores the prior) gives us a result that is entirely determined by the raw counts (6 heads and 4 tails, say). In contrast the MAP estimate is regularized towards the prior mean, which is why the MAP estimate is somewhere between 0.5 and 0.6.

# Doing the same thing with AutoGuides

In the above we defined guides by hand. 
It's often much easier to rely on Pyro's [AutoGuide machinery](https://docs.pyro.ai/en/stable/infer.autoguide.html?highlight=autoguide). 
Let's see how we can do MLE and MAP inference using AutoGuides.

To do MLE estimation we first use [`mask(False)`](https://docs.pyro.ai/en/stable/poutine.html?highlight=mask#pyro.poutine.handlers.mask) to instruct Pyro to ignore the `log_prob` of the latent variable `latent_fairness` in the model. 
(Note we need to do this for every latent variable.)
This way the only non-zero `log_prob` in the model will be from the Bernoulli likelihood and ELBO maximization will be equivalent to likelihood maximization.

In [11]:
def masked_model(data):
    f = pyro.sample("latent_fairness", 
                    dist.Beta(10.0, 10.0).mask(False))
    with pyro.plate("data", data.size(0)):
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

Next we define an [`AutoDelta`](https://docs.pyro.ai/en/stable/infer.autoguide.html?highlight=autodelta#autodelta) guide, which learns a point estimate for each latent variable (i.e. we do not learn any uncertainty):

In [12]:
autoguide_mle = pyro.infer.autoguide.AutoDelta(masked_model)
train(masked_model, autoguide_mle)
print("Our MLE estimate of the latent fairness is {:.3f}".format(
      autoguide_mle.median(data)["latent_fairness"].item()))

[iter 0]  loss: 6.8326
[iter 50]  loss: 6.7373
[iter 100]  loss: 6.7301
[iter 150]  loss: 6.7301
[iter 200]  loss: 6.7301
Our MLE estimate of the latent fairness is 0.600


To do MAP inference we again use an `AutoDelta` guide but this time on the original model in which `latent_fairness` is a latent variable:

In [13]:
autoguide_map = pyro.infer.autoguide.AutoDelta(original_model)
train(original_model, autoguide_map)
print("Our MAP estimate of the latent fairness is {:.3f}".format(
      autoguide_map.median(data)["latent_fairness"].item()))

[iter 0]  loss: 5.6073
[iter 50]  loss: 5.6004
[iter 100]  loss: 5.6004
[iter 150]  loss: 5.6004
[iter 200]  loss: 5.6004
Our MAP estimate of the latent fairness is 0.536
